# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we are going to identify three most suitable neighborhoods in Toronto to open a new grocery store.

We target on selling high quality foods, personal care products and home utility to people who are willing to spend on more premium products. According to the statistics in 2019, the median household income in Toronto is 71,631 dollars. Therefore, It is preferred to choose **neighbourhoods with household income of 70,000 dollars or above and their percentage is greater than that of all households in Toronto**.

Among these neighbourhoods, we want to find out **five of them with the least competition for grocery stores, i.e. the least number of grocery stores per capita around the neighborhoods**. 

Safety and security are also concerns for running a stable business. We will further choose the **final three out of the five neighbourhoods with the least crime rates per capita related to Break and Enter, Robbery and Theft**.

## Data <a name="data"></a>

Based on the criteria mentioned above to tackle our business problem, the following data sources will be used to help us make the decision:
1. Household income of of neighbourhoods in Toronto <br>
Source: Neighborhood Profiles from City of Toronto's Open Data Portal<br> https://open.toronto.ca/dataset/neighbourhood-profiles/


2. Number of grocery stores per capita in neighbourhoods in Toronto <br>
Source: Foursquare API


3. Crime rates per capita of neighbourhoods in Toronto <br>
Source: Neigborhood Crime Rates 2014-2019 from Toronto Police Service <br>
https://data.torontopolice.on.ca/datasets/neighbourhood-crime-rates-boundary-file-

### Crime rates per capita of each neighborhood in 2014-2019 (Include only Break and Enter, Robbery, Theft Over)

Let's put the data from Toronto Police Service into dataframe

In [2]:
import pandas as pd

In [3]:
crime_rates_df=pd.read_csv("crime_rates.csv")
crime_rates_df.head()

,OBJECTID,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,...,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length
0,1,Yonge-St.Clair,97,12528,20,29,39,27,34,37,...,5,8,0,3,6,4.3,1.00,47.9,1.161315e+06,5873.270582
1,2,York University Heights,27,27593,271,296,361,344,357,370,...,46,37,39,38,28,36.3,-0.26,101.5,1.324666e+07,18504.777326
2,3,Lansing-Westgate,38,16164,44,80,68,85,75,72,...,5,5,11,6,11,7.0,0.83,68.1,5.346186e+06,11112.109625
3,4,Yorkdale-Glen Park,31,14804,106,136,174,161,175,209,...,14,26,23,20,29,22.5,0.45,195.9,6.038326e+06,10079.426920
4,5,Stonegate-Queensway,16,25051,88,71,76,95,87,82,...,8,4,6,7,4,6.0,-0.43,16.0,7.946202e+06,11853.189878


We select the column of average crime rate of Break and Enter, Robbery and Theft Over, and the population

In [4]:
crime_rates_df2 = crime_rates_df[["Neighbourhood","Population", "BreakandEnter_AVG", "Robbery_AVG", "TheftOver_AVG"]]
crime_rates_df2.head()

,Neighbourhood,Population,BreakandEnter_AVG,Robbery_AVG,TheftOver_AVG
0,Yonge-St.Clair,12528,23.3,5.7,4.3
1,York University Heights,27593,113.2,75.8,36.3
2,Lansing-Westgate,16164,38.8,14.7,7.0
3,Yorkdale-Glen Park,14804,63.3,31.5,22.5
4,Stonegate-Queensway,25051,52.8,20.7,6.0


We calculate the sum of these 3 categories as the total crime rates and then divide it by the population to find out the crime rate per capita of each neighbourhood

In [5]:
crime_rates_df2["Total_crime_rate"] = crime_rates_df2.iloc[:, 2:5].sum(axis=1)
crime_rates_df2["Total_crime_rate_per_capita"] = crime_rates_df2["Total_crime_rate"] / crime_rates_df2["Population"] 
crime_rates_df2.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighbourhood,Population,BreakandEnter_AVG,Robbery_AVG,TheftOver_AVG,Total_crime_rate,Total_crime_rate_per_capita
0,Yonge-St.Clair,12528,23.3,5.7,4.3,33.3,0.002658
1,York University Heights,27593,113.2,75.8,36.3,225.3,0.008165
2,Lansing-Westgate,16164,38.8,14.7,7.0,60.5,0.003743
3,Yorkdale-Glen Park,14804,63.3,31.5,22.5,117.3,0.007924
4,Stonegate-Queensway,25051,52.8,20.7,6.0,79.5,0.003174


### Percentage of household income over 70k in each neighborhood

We read the data of neighbourhood profiles from City of Toronto's Open Data Portal into a dataframe

In [7]:
neigh_profile=pd.read_csv("neighborhood_profiles_2016b.csv")
neigh_profile.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


This is a long list with other irrelevant information. We select only the rows showing the househould income.

In [12]:
household_df=neigh_profile.iloc[1054:1071, 4:]
household_df.head()

,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
1054,Total - Household after-tax income groups in 2...,"1,112,930","9,115","8,140","4,620","15,940","12,125","6,090","15,070","9,525",...,"7,550","8,510","18,430","5,455","3,450","5,880","5,680","7,010","10,165","5,345"
1055,"Under $5,000","34,795",150,320,70,905,300,130,"2,545",610,...,575,105,435,70,60,135,215,240,350,115
1056,"$5,000 to $9,999","23,615",105,130,35,485,150,85,740,255,...,215,80,470,125,60,125,105,125,220,70
1057,"$10,000 to $14,999","37,080",165,205,80,660,245,150,745,290,...,265,155,685,260,110,185,140,190,350,120
1058,"$15,000 to $19,999","55,930",450,275,85,785,435,430,720,355,...,610,305,"1,160",325,200,405,190,220,540,225


We need to transpose the dataframe as other dataframes having the neighbourhoods as the rows.

In [13]:
household_df2 = household_df.transpose()
household_df2.head()

,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070
Characteristic,Total - Household after-tax income groups in 2...,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over"
City of Toronto,"1,112,930","34,795","23,615","37,080","55,930","51,910","52,205","53,515","55,545","55,770","53,385","98,910","86,755","75,120","63,360","51,570","263,465"
Agincourt North,"9,115",150,105,165,450,340,565,450,510,455,475,930,755,695,640,500,"1,920"
Agincourt South-Malvern West,"8,140",320,130,205,275,350,415,395,450,470,445,860,755,630,500,405,"1,500"
Alderwood,"4,620",70,35,80,85,160,175,170,210,185,225,390,355,365,310,265,"1,520"


As we target only household income over 70000 dollars, we select suitable columns to reflect this and also keep the total number of household for further calculation

In [14]:
household_df3=household_df2.drop(household_df2.columns[1:13],axis=1)
household_df3.head()

,1054,1067,1068,1069,1070
Characteristic,Total - Household after-tax income groups in 2...,"$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over"
City of Toronto,"1,112,930","75,120","63,360","51,570","263,465"
Agincourt North,"9,115",695,640,500,"1,920"
Agincourt South-Malvern West,"8,140",630,500,405,"1,500"
Alderwood,"4,620",365,310,265,"1,520"


As the format of the first row is not what we want, we drop that row and rebuild the index with the corresponding names

In [15]:
household_df3.columns = ['total', '70000-79999','80000-89999','90000-99999','over 100000']

In [21]:
household_df4 = household_df3.drop('Characteristic')
household_df4.head()

,total,70000-79999,80000-89999,90000-99999,over 100000
City of Toronto,"1,112,930","75,120","63,360","51,570","263,465"
Agincourt North,"9,115",695,640,500,"1,920"
Agincourt South-Malvern West,"8,140",630,500,405,"1,500"
Alderwood,"4,620",365,310,265,"1,520"
Annex,"15,940",820,750,620,"4,755"


The numeric values contain the thousand separator (,), so we need to remove this before we can change the data type to float for mathematical calculation

In [18]:
def convert_comma(val):
    """
    Convert the string number value to a float
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','')
    return float(new_val)

In [22]:
household_df4["total"] = household_df4["total"].apply(convert_comma)
household_df4["70000-79999"] = household_df4["70000-79999"].apply(convert_comma)
household_df4["80000-89999"] = household_df4["80000-89999"].apply(convert_comma)
household_df4["90000-99999"] = household_df4["90000-99999"].apply(convert_comma)
household_df4["over 100000"] = household_df4["over 100000"].apply(convert_comma)
household_df4.head()

,total,70000-79999,80000-89999,90000-99999,over 100000
City of Toronto,1112930.0,75120.0,63360.0,51570.0,263465.0
Agincourt North,9115.0,695.0,640.0,500.0,1920.0
Agincourt South-Malvern West,8140.0,630.0,500.0,405.0,1500.0
Alderwood,4620.0,365.0,310.0,265.0,1520.0
Annex,15940.0,820.0,750.0,620.0,4755.0


We calculate the sum of the household with income over 70k and then find out the percentage of these household in each neighbourhood and the city of Toronto

In [23]:
household_df4['Total over 70k']=household_df4.iloc[:, 1:6].sum(axis=1)
household_df4['Percentage_over_70k']=household_df4['Total over 70k']/household_df4['total']
household_df4.head()

,total,70000-79999,80000-89999,90000-99999,over 100000,Total over 70k,Percentage_over_70k
City of Toronto,1112930.0,75120.0,63360.0,51570.0,263465.0,453515.0,0.407496
Agincourt North,9115.0,695.0,640.0,500.0,1920.0,3755.0,0.411958
Agincourt South-Malvern West,8140.0,630.0,500.0,405.0,1500.0,3035.0,0.372850
Alderwood,4620.0,365.0,310.0,265.0,1520.0,2460.0,0.532468
Annex,15940.0,820.0,750.0,620.0,4755.0,6945.0,0.435696


We will use the percentage over 70k of City of Toronto (0.407496) as one of the criteria in the final section, but now we will drop that row for merging other dataframe later

In [24]:
household_df5 = household_df4.drop('City of Toronto')
household_df5.head()

,total,70000-79999,80000-89999,90000-99999,over 100000,Total over 70k,Percentage_over_70k
Agincourt North,9115.0,695.0,640.0,500.0,1920.0,3755.0,0.411958
Agincourt South-Malvern West,8140.0,630.0,500.0,405.0,1500.0,3035.0,0.372850
Alderwood,4620.0,365.0,310.0,265.0,1520.0,2460.0,0.532468
Annex,15940.0,820.0,750.0,620.0,4755.0,6945.0,0.435696
Banbury-Don Mills,12125.0,790.0,690.0,595.0,3600.0,5675.0,0.468041


Adding the index name for merging later on.

In [26]:
household_df5.index.name='Neighbourhood'
household_df5.head()

,total,70000-79999,80000-89999,90000-99999,over 100000,Total over 70k,Percentage_over_70k
Neighbourhood,,,,,,,
Agincourt North,9115.0,695.0,640.0,500.0,1920.0,3755.0,0.411958
Agincourt South-Malvern West,8140.0,630.0,500.0,405.0,1500.0,3035.0,0.372850
Alderwood,4620.0,365.0,310.0,265.0,1520.0,2460.0,0.532468
Annex,15940.0,820.0,750.0,620.0,4755.0,6945.0,0.435696
Banbury-Don Mills,12125.0,790.0,690.0,595.0,3600.0,5675.0,0.468041


We now have the crime rate per capita and the percentage of household over 70k, we can now merge the two dataframe by neighbourhood names

In [29]:
df_merged=pd.merge(household_df5,crime_rates_df2, on="Neighbourhood")
df_merged.head()

,Neighbourhood,total,70000-79999,80000-89999,90000-99999,over 100000,Total over 70k,Percentage_over_70k,Population,BreakandEnter_AVG,Robbery_AVG,TheftOver_AVG,Total_crime_rate,Total_crime_rate_per_capita
0,Agincourt North,9115.0,695.0,640.0,500.0,1920.0,3755.0,0.411958,29113,53.5,30.2,4.7,88.4,0.003036
1,Agincourt South-Malvern West,8140.0,630.0,500.0,405.0,1500.0,3035.0,0.372850,23757,79.8,27.3,13.3,120.4,0.005068
2,Alderwood,4620.0,365.0,310.0,265.0,1520.0,2460.0,0.532468,12054,24.7,6.8,6.8,38.3,0.003177
3,Annex,15940.0,820.0,750.0,620.0,4755.0,6945.0,0.435696,30526,147.5,40.8,29.5,217.8,0.007135
4,Banbury-Don Mills,12125.0,790.0,690.0,595.0,3600.0,5675.0,0.468041,27695,73.2,15.0,10.3,98.5,0.003557


We drop the columns we don't need any more.

In [30]:
df_merged2=df_merged.drop(columns=['total',  '70000-79999', '80000-89999', '90000-99999','over 100000', 'Total over 70k', 'BreakandEnter_AVG', 'Robbery_AVG', 'TheftOver_AVG'])
df_merged2.head()

,Neighbourhood,Percentage_over_70k,Population,Total_crime_rate,Total_crime_rate_per_capita
0,Agincourt North,0.411958,29113,88.4,0.003036
1,Agincourt South-Malvern West,0.372850,23757,120.4,0.005068
2,Alderwood,0.532468,12054,38.3,0.003177
3,Annex,0.435696,30526,217.8,0.007135
4,Banbury-Don Mills,0.468041,27695,98.5,0.003557


### Number of grocery stores per capita in each neighbourhood

We have tried to use geopy to find out the coordinates with the name of neighbourhood by using a for loop. However, geopy cannot identify some of the names and failed to return the coordinates. This makes loop cannot not be completed. We have tried different ways for many hours but no luck. Therefore we manually entered the coordinates in a csv file instead.

Let's read the csv into dataframe

In [31]:
coordinates=pd.read_csv("coordinates.csv")
coordinates.head()

,Neighbourhood,Latitude,longitude
0,Agincourt North,43.808038,-79.266439
1,Agincourt South-Malvern West,43.785353,-79.278549
2,Alderwood,43.601717,-79.545232
3,Annex,43.670338,-79.407117
4,Banbury-Don Mills,43.742796,-79.369957


We merge the coordinates to the previous merged dataframe.

In [36]:
df_merged3=pd.merge(df_merged2,coordinates, on="Neighbourhood")
df_merged3.head()

,Neighbourhood,Percentage_over_70k,Population,Total_crime_rate,Total_crime_rate_per_capita,Latitude,longitude
0,Agincourt North,0.411958,29113,88.4,0.003036,43.808038,-79.266439
1,Agincourt South-Malvern West,0.372850,23757,120.4,0.005068,43.785353,-79.278549
2,Alderwood,0.532468,12054,38.3,0.003177,43.601717,-79.545232
3,Annex,0.435696,30526,217.8,0.007135,43.670338,-79.407117
4,Banbury-Don Mills,0.468041,27695,98.5,0.003557,43.742796,-79.369957


Let's find out the grocery stores in each neighbourhood by using Foursqaure API. We specify to search grocery store with the category ID.

In [39]:
import requests
CLIENT_ID = 'I34N2FUJWRUUAQOI5PPZ5B2ROPLBWLGLUFK3JIKKQOD3RA4F' 
CLIENT_SECRET = 'V2KV1BMXEFIDV2JMFJTKZKGNTVWGWOZGT0I5KK4RHPPCOUXE' 
VERSION = '20180605' 
CAT_ID = '4bf58dd8d48988d118951735'

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CAT_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood_Latitude', 
                  'Neighbourhood_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

In [41]:
toronto_venues = getNearbyVenues(names=df_merged3['Neighbourhood'],
                                   latitudes=df_merged3['Latitude'],
                                   longitudes=df_merged3['longitude']
                                  )

Agincourt North
Agincourt South-Malvern West
Alderwood
Annex
Banbury-Don Mills
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods-Steeles
Bedford Park-Nortown
Beechborough-Greenbrook
Bendale
Birchcliffe-Cliffside
Black Creek
Blake-Jones
Briar Hill-Belgravia
Bridle Path-Sunnybrook-York Mills
Broadview North
Brookhaven-Amesbury
Cabbagetown-South St.James Town
Caledonia-Fairbank
Casa Loma
Centennial Scarborough
Church-Yonge Corridor
Clairlea-Birchmount
Clanton Park
Cliffcrest
Corso Italia-Davenport
Danforth
Danforth East York
Don Valley Village
Dorset Park
Dovercourt-Wallace Emerson-Junction
Downsview-Roding-CFB
Dufferin Grove
East End-Danforth
Edenbridge-Humber Valley
Eglinton East
Elms-Old Rexdale
Englemount-Lawrence
Eringate-Centennial-West Deane
Etobicoke West Mall
Flemingdon Park
Forest Hill North
Forest Hill South
Glenfield-Jane Heights
Greenwood-Coxwell
Guildwood
Henry Farm
High Park North
High Park-Swansea
Highland Creek
Hillcrest Village
Humber Heights-Westmount
Hum

In [42]:
toronto_venues.head()

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Agincourt North,43.808038,-79.266439,Bulk Heaven,43.810355,-79.269789,Grocery Store
1,Agincourt South-Malvern West,43.785353,-79.278549,Lam Shing Trading Co.,43.785398,-79.278149,Grocery Store
2,Annex,43.670338,-79.407117,Noah's Natural Food,43.666915,-79.403458,Grocery Store
3,Annex,43.670338,-79.407117,food plus market 24 hrs,43.674515,-79.407138,Grocery Store
4,Annex,43.670338,-79.407117,Bloor Superfresh,43.666147,-79.406601,Grocery Store


We count the number of grocery store by using groupby and count.

In [45]:
grocery_store_grouped= toronto_venues.groupby(["Neighbourhood"]).count()
grocery_store_grouped2=grocery_store_grouped[["Venue_Category"]]
grocery_store_grouped2.columns = ['Number_of_grocery_stores']
grocery_store_grouped2.head()

,Number_of_grocery_stores
Neighbourhood,
Agincourt North,1
Agincourt South-Malvern West,1
Annex,5
Bay Street Corridor,9
Beechborough-Greenbrook,1


Now we merge the column of number of grocery store to the dataframe.

In [46]:
df_all=pd.merge(df_merged3,grocery_store_grouped2, on="Neighbourhood")
df_all.head()

,Neighbourhood,Percentage_over_70k,Population,Total_crime_rate,Total_crime_rate_per_capita,Latitude,longitude,Number_of_grocery_stores
0,Agincourt North,0.411958,29113,88.4,0.003036,43.808038,-79.266439,1
1,Agincourt South-Malvern West,0.372850,23757,120.4,0.005068,43.785353,-79.278549,1
2,Annex,0.435696,30526,217.8,0.007135,43.670338,-79.407117,5
3,Bay Street Corridor,0.311214,25797,332.3,0.012881,43.667342,-79.388457,9
4,Beechborough-Greenbrook,0.247170,6577,31.1,0.004729,43.695030,-79.471683,1


We calculate the number of grocery stores per capita

In [47]:
df_all["grocery_stores_per_capita"]=df_all["Number_of_grocery_stores"]/df_all["Population"]
df_all.head()

,Neighbourhood,Percentage_over_70k,Population,Total_crime_rate,Total_crime_rate_per_capita,Latitude,longitude,Number_of_grocery_stores,grocery_stores_per_capita
0,Agincourt North,0.411958,29113,88.4,0.003036,43.808038,-79.266439,1,0.000034
1,Agincourt South-Malvern West,0.372850,23757,120.4,0.005068,43.785353,-79.278549,1,0.000042
2,Annex,0.435696,30526,217.8,0.007135,43.670338,-79.407117,5,0.000164
3,Bay Street Corridor,0.311214,25797,332.3,0.012881,43.667342,-79.388457,9,0.000349
4,Beechborough-Greenbrook,0.247170,6577,31.1,0.004729,43.695030,-79.471683,1,0.000152


## Methodology

In this project we try to find the top three neighbourhoods for opening a grocery store in Toronto. Such neighbourhoods should have a high percentage of households with income over 70,000 dollars, low number of grocery stores per capita and low crime rates per capita. <br>
<br>
Now we have collected all the data for further analysis in the data section. In analysis section, we will screen out using these data based on the criteria we have mentioned in the introdction section. <br>
<br>
Fristly, we will find out the all the neighbourhoods where the pecentage of households with incomes over 70,000 dollars greater than that of the whole city of Toronto. Among these neighbourhoods, we will find out five of them with the lowest number of grocery stores per capita. Finally, we will then select three out of these five neighbourhoods with the least crime rate per capita as our choices.

# Analysis

### Making the final decision of the top 3 neighbourhood for opening a new grocery store with our raw data

Recall the percentage of household income over 70k of City of Toronto is 0.407496. We use it as the criteria to select the neighbourhood.

In [48]:
df_all2=df_all[df_all.Percentage_over_70k > 0.407496]
df_all2.head()

,Neighbourhood,Percentage_over_70k,Population,Total_crime_rate,Total_crime_rate_per_capita,Latitude,longitude,Number_of_grocery_stores,grocery_stores_per_capita
0,Agincourt North,0.411958,29113,88.4,0.003036,43.808038,-79.266439,1,0.000034
2,Annex,0.435696,30526,217.8,0.007135,43.670338,-79.407117,5,0.000164
6,Birchcliffe-Cliffside,0.461371,22291,99.5,0.004464,43.711170,-79.248177,1,0.000045
11,Caledonia-Fairbank,0.413043,9955,31.1,0.003124,43.692370,-79.465356,3,0.000301
12,Casa Loma,0.545791,10968,36.1,0.003291,43.678102,-79.409416,2,0.000182


We then find out the 5 neighbourhood withe the least grocery stores per capita

In [49]:
df_all3=df_all2.sort_values(['grocery_stores_per_capita'], ascending=[True])
df_all3.head(5)

,Neighbourhood,Percentage_over_70k,Population,Total_crime_rate,Total_crime_rate_per_capita,Latitude,longitude,Number_of_grocery_stores,grocery_stores_per_capita
74,West Humber-Clairville,0.440233,33312,281.8,0.008459,43.735781,-79.625865,1,0.000030
0,Agincourt North,0.411958,29113,88.4,0.003036,43.808038,-79.266439,1,0.000034
14,Clairlea-Birchmount,0.425866,26984,146.9,0.005444,43.708823,-79.295986,1,0.000037
26,High Park-Swansea,0.500000,23925,81.5,0.003406,43.653867,-79.466864,1,0.000042
6,Birchcliffe-Cliffside,0.461371,22291,99.5,0.004464,43.711170,-79.248177,1,0.000045


We further find out the 3 neighbourhoods with the least crime rate per capita

In [50]:
df_all4=df_all3.head(5)
df_all5=df_all4.sort_values(['Total_crime_rate_per_capita'], ascending=[True])
df_all5

,Neighbourhood,Percentage_over_70k,Population,Total_crime_rate,Total_crime_rate_per_capita,Latitude,longitude,Number_of_grocery_stores,grocery_stores_per_capita
0,Agincourt North,0.411958,29113,88.4,0.003036,43.808038,-79.266439,1,0.000034
26,High Park-Swansea,0.500000,23925,81.5,0.003406,43.653867,-79.466864,1,0.000042
6,Birchcliffe-Cliffside,0.461371,22291,99.5,0.004464,43.711170,-79.248177,1,0.000045
14,Clairlea-Birchmount,0.425866,26984,146.9,0.005444,43.708823,-79.295986,1,0.000037
74,West Humber-Clairville,0.440233,33312,281.8,0.008459,43.735781,-79.625865,1,0.000030


We now obtained the final top 3 neighbourhoods for opening our new grocery store are **Agincourt North, High Park-Swansea and Birchcliffe-Cliffside**.

## Results and Discussion

Our analysis has shown that Agincourt North, High Park-Swansea and Birchcliffe-Cliffside are the best neighbourhoods in Toronto for opening our grocery store. These neighbourhoods have a higher percentage of households with income over 70,000 dollars compared to the average of Toronto, low number of grocery stores and low crime rate per capita.<br>
<br>
It is worth noting that although we know the household incomes matching with our target customer, we do not have information about the customer’s background and their choices on products in these neighbourhoods. This can greatly affect what kind of products we should sell in our store. Also, we still need more research on the locations in which more people go for shopping as this has not been reflected in our project.


## Conclusion

The purpose of this project is to find out the top 3 neighbourhoods to open the grocery store matching with our criteria, namely Agincourt North, High Park-Swansea and Birchcliffe-Cliffside. Although this helps to narrow down the choices, it still needs further research on the customer's choice on products and the exact locations suitable for opening the store within these neighbourhoods.